In [70]:
#%matplotlib inline
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt

In [71]:
#import seaborn as sns
#sns.set(color_codes=True)

In the following problems we will code two different approaches for solving linear regression problems and compare how they scale as a function of the dimensionality of the data. We will also investigate the effects of linear and non-linear features in the predictions made by linear models.

We will be working with the regression data set Protein Tertiary Structure: <a href='https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv'>Click me!</a>. This data set contains information about predicted conformations for 45730 proteins. 

In the data, the target variable y is the root-mean-square deviation (RMSD) of the predicted conformations with respect to the true properly folded form of the protein. The RMSD is the measure of the average distance between the atoms (usually the backbone atoms) of superimposed proteins. The features x are physico-chemical properties of the proteins in their true foldedform. After downloading the file CASP.csv we can load the data into python using

In [72]:
data = np.loadtxt('CASP.csv', delimiter = ',', skiprows = 1)

We can then obtain the vector of target variables and the feature matrix using

In [73]:
y = data[ : , 0 ]
X = data[ : , 1 : ]

We can then split the original data into a training set with 90% of the data entries in the file CASP.csv and a
test set with the remaining 10% of the entries. Normally, the spliting of the data is done at random, but here
we ask you to put into the training set the first 90% of the elements from the file CASP.csv so that we can
verify that the values that you will be reporting are correct. (This should not cause problems, because the
rows of the file are in a random order.) The following python function splits the data as requested:

In [74]:
def split_train_test(X, y, fraction_train = 9.0 / 10.0):
    end_train = round(X.shape[ 0 ] * fraction_train)
    X_train = X[ 0 : end_train, ]
    y_train = y[ 0 : end_train ]
    X_test = X[ end_train :, ]
    y_test = y[ end_train : ]
    return X_train, y_train, X_test, y_test

We can then normalize the features so that they have zero mean and unit deviation in the training set. This
is a standard step before the application of many machine learning methods. We can use the following
python function to perform this operation:

In [75]:
def normalize_features(X_train, X_test):
    mean_X_train = np.mean(X_train, 0)
    std_X_train = np.std(X_train, 0)
    std_X_train[ std_X_train == 0 ] = 1
    X_train_normalized = (X_train - mean_X_train) / std_X_train
    X_test_normalized = (X_test - mean_X_train) / std_X_train
    return X_train_normalized, X_test_normalized

After these steps are done, we can concatenate a bias feature (one feature which always takes value 1) to
the observations in the normalized training and test sets:

In [76]:
X_train, y_train, X_test, y_test = split_train_test(X, y)
X_train, X_test = normalize_features(X_train, X_test)
X_train = np.concatenate((X_train, np.ones((X_train.shape[ 0 ], 1))), 1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[ 0 ], 1))), 1)

We are now ready to apply our machine learning methods to the normalized training set and evaluate their performance on the normalized test set. In the following problems, you will be asked to report some numbers and produce some figures. Include these numbers and figures in your assignment report. The
numbers should be reported with up to 8 decimals.

##Problem 4

Assume that the targets $y$ are obtained as a function of the normalized features $x$ according to a Bayesian linear model with additive Gaussian noise with variance $\sigma^2 = 1.0$ and a Gaussian prior on the regression coefficients $w$ with precision matrix $\tau^{−2}I$ where $\tau^{−2} = 10$. Code a routine that finds the Maximum a Posteriori (MAP) value $\hat{w}$ for $w$ given the normalized training data using the QR decomposition (see Section 7.5.2 in Murphy’s book).

- Report the value of $\hat{w}$ obtained.
- Report the root mean squared error (RMSE) of $\hat{w}$ in the normalized test set.

##Task 5

Write code that evaluates the log-posterior probability (up to an additive constant) of $w$ according to the previous Bayesian linear model and the normalized training data. Write also code that evaluates the gradient of the log-posterior probability with respect to $w$. Merge the two pieces of code into a function called <i>obj_and_gradient</i>, that for a value of $w$, returns the log-posterior probability value and the corresponding gradient. Verify that the gradient computation is correct using the approximation 


$\frac{d\,f(x)}{d\,x}\approx\frac{f(x+\epsilon)-f(x-\epsilon)}{2\epsilon}$


Write then a function that uses ”obj_and_gradient” to find the MAP solution $\hat{w}$ for $w$ by running 100 iterations of the L-BFGS numerical optimization method with $0$ as the initial point for the optimization. The L-BFGS method is an iterative method for solving nonlinear optimization problems.

You will learn more about numerical optimization and about this method in one of the sections of this course. However, for this problem you can use the method as a black box that returns the MAP solution by sequentially evaluating the objective function and its gradient for different input values. In python, a variant of the L-BFGS method called L-BFGS-B is implemented in scipy.optimize.minimize. Since scipy.optimize.minimize only minimizes, you can work with the negative log-posterior probability to transform the maximization problem into a minimization one. 

- Report the value of $\hat{w}$ obtained.
- Report the RMSE of the predictions made with $\hat{w}$ in the normalized test set.

##Task 6

Linear regression can be extended to model non-linear relationships by replacing the original features $x$ with some non-linear functions of the original features $\phi(x)$. We can automatically generate one such non-linear function by sampling a random weight vector $a\sim\mathcal{N}(0,I)$ and a corresponding random bias $b\sim U[0,2\pi]$ and then making $\phi(x)=\cos(a^{T}x+b)$. By repeating this process $d$ times we can generate $d$ non-linear functions that, when applied to the original features, produce a non-linear mapping of the data into a new $d$ dimensional space. We can encode these $d$ functions into a matrix $A$ with $d$ rows, each one with the weights for each function, and a $d$-dimensional vector $b$ with the biases for each function. The new mapped fetures are then obtained as $\phi(x)=\cos(Ax+b)$, where cos applied to a vector returns another vector whose elements are the result of applying cos to the individual elements of the original vector.

Generate 4 sets of non-linear functions, each one with $d$ = 100, 200, 400, 600 functions, respectively, and use them to map the features in the original normalized training and test sets into 4 new feature spaces, each one of dimensionality given by the value of $d$. After this, for each value of $d$, find the MAP solution $\hat{w}$ for $w$ using the corresponding new training set and the method from problem 4. Use the same values for $\sigma^{2}$ and $\tau^{−2}$ as before. You are also asked to record the time taken by the method QR to obtain a value for $\hat{w}$. In python you can compute the time taken by a routine using the time package:

> import time

>    time_start = time.time()

>    routine_to_call()
 
>    running_time = time.time() - time_start

Next, compute the RMSE of the resulting predictor in the normalized test set. Repeat this process with the method from problem 5 (L-BFGS). 

-  Report the test RMSE obtained by each method for each value of $d$.

You are asked to generate a plot with the results obtained by each method (QR and L-BFGS) for each value of $d$. In this plot the $x$ axis should represent the time taken by each method to run and the $y$ axis should be the RMSE of the resulting predictor in the original test set before normalization. The plot should contain 4 points in red, representing the results obtained by the method QR for each value of $d$, and 4 points in blue, representing the results obtained by the method L-BFGS for each value of d.
Answer the following questions:

- Do the non-linear transformations help to reduce the prediction error? Why?
- What method (QR or L-BFGS) is faster? Why?